# 分类学习
## 决策树 Decision Tree

In [16]:
# 绘制Mermaid图表的工具，来自 https://gist.github.com/MLKrisJohnson/2d2df47879ee6afd3be9d6788241fe99
import base64
from IPython.display import Image, display

def mm_ink(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  return "https://mermaid.ink/img/" + base64_string

def mm_display(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, display it."""
  display(Image(url=mm_ink(graphbytes)))

def mm(graph):
  """Given a string containing a Mermaid-format graph, display it."""
  graphbytes = graph.encode("ascii")
  mm_display(graphbytes)

def mm_link(graph):
  """Given a string containing a Mermaid-format graph, return URL for display."""
  graphbytes = graph.encode("ascii")
  return mm_ink(graphbytes)
  
def mm_path(path):
  """Given a path to a file containing a Mermaid-format graph, display it"""
  with open(path, 'rb') as f:
    graphbytes = f.read()
  mm_display(graphbytes)

### 概要
决策树是主要用于分类和回归任务的数据结构，它由节点和连接这两部分组成。
 - 节点：每个节点都代表了一个数据标签
 - 连接：由父节点连接到子节点的标签特征。
我们下面会用一个简单的样例来解释决策树。

首先，我们需要一些数据，这些数据应该具备自己的数据标签和观察到的特征。这里我们以“学生是否通过考试”为案例，示例如下：
| 学习时长(StudyDuration:H/M/L) | 是否复习(Reviewed:Y/N) | 睡眠质量(SleepQuality:G/A(verage)/P(oor)) | 是否通过考试(Pass/Fail) |
|--------|--------|--------|-----------|
| 高     | 是    | 好     | 是        |
| 高     | 否    | 好     | 否        |
| 中     | 是    | 一般   | 是        |
| 低     | 否    | 差     | 否        |
| 低     | 是    | 好     | 否        |
| 低     | 是    | 一般   | 是        |
| 中     | 否    | 差     | 否        |
| 高     | 是    | 一般   | 是        |

随后，我们需要选取**特征**和**目标变量**。
 - 特征：特征指的是我们用来进行判断决策的指标，比如上表中的学习时长，是否复习等
 - 目标变量：目标变量指的是我们需要预测的指标，只能有一个，就是上表里的是否通过考试
接下来我们需要对这张数据表进行观察并构建一棵决策树。构建决策树时我们需要使用一些算法来确定应该以什么特征进行判断决策，但在这里先暂时忽略，就以最基础的直觉进行构建，如下：

In [25]:
with open('分类学习/决策树1.data', 'r', encoding='utf-8') as file:
    mm(file.read())

在这里，我们展示了一个纯粹直觉构建的决策树图表。当我们拿到一个新的数据，比如下表时，就可以根据上面构建的决策树进行预测。
| 学习时长 | 是否复习 | 睡眠质量 | 是否通过考试 |
|--------|--------|--------|-----------|
| 低     | 否    | 好     | ？        |

我们根据新的数据和决策树，首先判断学习时长，于是走L路线，然后判断是否复习，走H路线，随后判断睡眠质量，走G路线，最后得到的结果就直接是G边对应的叶子节点，也即Fail。于是我们就可以预测在这种情况下这位考生不能通过考试。

In [26]:
with open('分类学习/决策树2.data', 'r', encoding='utf-8') as file:
    mm(file.read())

## ID3算法（第三代迭代二叉树算法）
### 概述
我们在实际的模型学习里当然不可能靠人的直觉来判断决策树应该以什么边和节点来构建，所以我们必须要有一个算法来代替直觉找到“最符合数据分布”的决策树。在这里，我们会使用ID3算法来进行处理。

迭代二叉树算法的计算原理是对整个数据集进行“分裂”，也就是基于标签能提供的信息增益对整个数据集进行拆分。在选择出信息增益最大的标签之后，就会将数据集按这个标签进行分割，随后就像二叉树算法一样，对分割后的每一个子数据集重新执行这个算法，一直执行到所有标签全部被用来分割过一遍，或者达到预设的其他终止条件为止。

### 熵 Entropy
在上文中我们提到了**信息增益**这个关键词，在ID3算法里，信息增益指的就是数据集的信息熵。

从直观上理解，信息熵可以这么解释：当一个数据集里元素分布得越均匀时，这个数据集的熵就越高。反之，如果这个数据集里元素分布得不均匀，那么这个数据集的熵就越低。极端情况是，如果一个数据集中所有元素都分布在同一个类别，那么这个数据集的熵就是0，也就是熵此时最低。

